In [1]:
import os
import pandas as pd
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain.chat_models import ChatOpenAI
import re

In [10]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from operator import itemgetter
from langchain_experimental.tools import PythonAstREPLTool

from langchain_core.messages import ToolMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough

import pandas as pd



In [5]:

# Load CSV files
import pandas as pd
import os
from typing import List, Dict

def load_csv_files(file_paths: List[str]) -> Dict[str, pd.DataFrame]:
    file_data = {}
    
    for path in file_paths:
        file_name = os.path.basename(path)
        file_name_without_ext = os.path.splitext(file_name)[0]
        try:
            if path.endswith('.csv'):
                df = pd.read_csv(path)
            elif path.endswith('.xlsx') or path.endswith('.xls'):
                df = pd.read_excel(path)
            elif path.endswith('.json'):
                df = pd.read_json(path)
            else:
                print(f"Unsupported file format: {path}")
                continue
            file_data[file_name_without_ext] = df
        except Exception as e:
            print(f"Error reading {file_name}: {e}")
    
    return file_data

# Simulating file upload
uploaded_files = [
    "./data/Fib.csv",
"./data/Obstructive Lung Diseases.csv"
]
dfs = load_csv_files(uploaded_files)


In [6]:
df_1 = dfs['Fib']
df_2 = dfs['Obstructive Lung Diseases']

In [8]:
# llm = ChatOpenAI()
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [15]:
# tool = PythonAstREPLTool(locals={"df_1": df_1, "df_2": df_2})
from langchain_experimental.tools import PythonREPLTool
tools = [PythonREPLTool()]

from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI

instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
# llm.bind_tools(tools=[tool], tool_choice=tool.name)

PythonREPLTool()

In [ ]:
tool = PythonAstREPLTool(locals={"df_1": df_1, "df_2": df_2})
llm_with_tool = llm.bind_tools(tools=[tool], tool_choice=tool.name)
# df_template = """```python
# {df_name}.head().to_markdown()
# >>> {df_head}
# ```"""
# df_context = "\n\n".join(
#     df_template.format(df_head=_df.head().to_markdown(), df_name=df_name)
#     for _df, df_name in [(df_1, "df_1"), (df_2, "df_2")]
# )

# system = f"""You have access to a number of pandas dataframes. \
# Here is a sample of rows from each dataframe and the python code that was used to generate the sample:

# {df_context}

# Given a user question about the dataframes, write the Python code to answer it. \
# Don't assume you have access to any libraries other than built-in Python ones and pandas. \
# Make sure to refer only to the variables mentioned above."""
# prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

# chain = prompt | llm_with_tool | parser | tool
# chain.invoke(
#     {
#         "question": "return the difference in the correlation between age and fare and the correlation between fare and survival"
#     }
# )

In [16]:
agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    uploaded_files,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)
agent.run("how many rows in the age column are different between the two dfs?")

NameError: name 'create_csv_agent' is not defined

In [33]:
df_name=list(dfs.keys())
df1_rows= dfs['Fib'].head(5)
df2_rows=dfs['Obstructive Lung Diseases'].head(5)
df1= dfs['Fib']
df2=dfs['Obstructive Lung Diseases']

In [38]:
print(f"""You are povided with the following dataframes: {df_name}.
# The first dataframe has the following first 5 rows:{df1_rows},/n\n
# The second dataframe has the following first 5 rows: {df2_rows}""")

You are povided with the following dataframes: ['Fib', 'Obstructive Lung Diseases'].
# The first dataframe has the following first 5 rows:              Country  Driver Patient Population  \
0      United States**   Core               High   
1              China**   Core               High   
2                Japan   Core               High   
3  Russian Federation*   Core             Medium   
4               France   Core               High   

  IPF Diagnosed Incident Cases (2021)  \
0                              34,723   
1                              60,586   
2                              16,564   
3                              12,443   
4                               6,979   

   IPF Diagnosed Incident Cases Rate per 100k (2021) Competition  \
0                                               0.21      Medium   
1                                               0.09        High   
2                                               0.21        High   
3                             

In [43]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
# Define your custom prefix and suffix
custom_prefix = f"""You are povided with the following dataframes: {df_name}.
# The first dataframe has the following first 5 rows:{df1_rows},/n\n
# The second dataframe has the following first 5 rows: {df2_rows}"""


# Create the pandas dataframe agent
agent = create_pandas_dataframe_agent(
    llm=llm,
    df=[df1,df2],
    # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    prefix=custom_prefix,
    # suffix=custom_suffix,
    include_df_in_prompt=True,
    number_of_head_rows=5,
    allow_dangerous_code=True
)

In [45]:
agent.invoke("Can you weigh Patient Population at 25%, Competition at 25%, and Country Operations at 50% for Obstructive Lung Diseases to give me the prioritized country list?")

{'input': 'Can you weigh Patient Population at 25%, Competition at 25%, and Country Operations at 50% for Obstructive Lung Diseases to give me the prioritized\xa0country\xa0list?',
 'output': 'It seems there was an issue with reading the dataframes from the clipboard. Could you please provide the dataframes in another format or share the data with me so that I can proceed with the calculation?'}

### CSV agent

In [46]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

In [50]:
uploaded_files

['./data/Fib.csv', './data/Obstructive Lung Diseases.csv']

In [74]:
# Test on some sample data
df = pd.DataFrame(
    {
        "city": ["Toronto", "Tokyo", "Berlin"],
        "population": [2930000, 13960000, 3645000],
    }
)
df.to_csv("./data/sample_data.csv",index=False)

In [75]:
agent = create_csv_agent(
    OpenAI(temperature=0),
    path=["./data/sample_data.csv"],
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    prefix=custom_prefix,
    allow_dangerous_code=True
)

In [76]:
agent.invoke("How many rows are there ?")




> Entering new AgentExecutor chain...


TypeError: Completions.create() got an unexpected keyword argument 'functions'

### sonet 3.5 Opus

In [79]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [ ]:
# Function to create an agent for a single CSV file
def create_single_csv_agent(file_path):
    return create_csv_agent(
        ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        file_path,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS
    )

# Function to create agents for multiple CSV files
def create_multi_csv_agents(file_paths):
    return [create_single_csv_agent(file_path) for file_path in file_paths]

# Function to process a query across multiple CSV files
def process_query(agents, query):
    responses = []
    for agent in agents:
        try:
            response = agent.run(query)
            responses.append(response)
        except Exception as e:
            responses.append(f"Error processing query: {str(e)}")
    return responses



In [ ]:
# Main function
def main():
    # List of CSV files
    csv_files = ['Fib.csv', 'Obstrcutive.csv']  # Add more CSV files as needed
    
    # Create agents for each CSV file
    agents = create_multi_csv_agents(csv_files)
    
    while True:
        # Get user query
        query = input("Enter your question (or 'quit' to exit): ")
        
        if query.lower() == 'quit':
            break
        
        # Process the query
        responses = process_query(agents, query)
        
        # Print responses
        print("\nResponses:")
        for i, response in enumerate(responses):
            print(f"\nFrom {csv_files[i]}:")
            print(response)

if __name__ == "__main__":
    main()